In [ ]:
import pandas as pd
from preProccesPuzzle import PreProcess
from clue_classifier import ClueClassifier
from constraints import Constraint, IdentityConstrain, NextToConstrain, DistanceConstrain,RightConstrain, LeftConstrain, DirectRightConstrain, DirectLeftConstrain, PositionAbsoluteConstrain, PositionAbsoluteNegativeConstrain
from constraint_solver import ConstraintSolver
from concurrent.futures import ProcessPoolExecutor
import time


In [ ]:
def constraint_factory(attrs, clues):
    constrains: list[Constraint] = []
    classifier = ClueClassifier()
    for c in clues:
        clue, clue_type = classifier.classify(c)

        if clue_type == "IDENTITY":
            constrains.append(IdentityConstrain(attrs, clue))
        if clue_type == "NEXT_TO":
            constrains.append(NextToConstrain(attrs, clue))
        if clue_type == "LEFT":
            constrains.append(LeftConstrain(attrs, clue))
        if clue_type == "RIGHT":
            constrains.append(RightConstrain(attrs, clue))
        if clue_type == "DISTANCE":
            constrains.append(DistanceConstrain(attrs, clue))
        if clue_type == "DIRECT_LEFT":
            constrains.append(DirectLeftConstrain(attrs, clue))
        if clue_type == "DIRECT_RIGHT":
            constrains.append(DirectRightConstrain(attrs, clue))
        if clue_type == "POSITION_ABSOLUTE":
            constrains.append(PositionAbsoluteConstrain(attrs, clue))
        if clue_type == "POSITION_ABSOLUTE_NEGATIVE":
            constrains.append(PositionAbsoluteNegativeConstrain(attrs, clue))
        if clue_type == "UNKNOWN":
            raise TypeError
    
    return constrains

In [ ]:
def convert_solver_solution_to_gridmode_format(solver_solution, attrs, gridmode_solution):
    header = gridmode_solution.get("header", [])
    rows = gridmode_solution.get("rows", [])
    
    # Create mapping by matching header (excluding 'House') with attrs keys in order
    attr_keys = list(attrs.keys())
    header_without_house = [h for h in header if h.lower() != 'house']
    
    attr_mapping = dict(zip(attr_keys, header_without_house))
    
    # Convert solver solution to gridmode format
    converted_rows = []
    sorted_positions = sorted(solver_solution.keys())
    
    for pos in sorted_positions:
        row = []
        attrs_for_pos = solver_solution[pos]
        
        for header_name in header:
            if header_name.lower() == "house":
                row.append(str(pos))
            else:
                # Find the attr_key for this header column
                attr_key = next((k for k, v in attr_mapping.items() if v == header_name), None)
                if attr_key:
                    value = attrs_for_pos.get(attr_key, "")
                    row.append(str(value))
                else:
                    row.append("")
        
        converted_rows.append(row)
    
    converted_solution = {
        "header": list(header),
        "rows": converted_rows
    }
    
    # Compare with ground truth
    is_correct = True
    mismatches = []
    
    if len(converted_rows) != len(rows):
        is_correct = False
        mismatches.append(f"Row count mismatch: expected {len(rows)}, got {len(converted_rows)}")
    
    for i, (converted_row, truth_row) in enumerate(zip(converted_rows, rows)):
        for j, (converted_val, truth_val) in enumerate(zip(converted_row, truth_row)):
            if converted_val.lower() != truth_val.lower():
                is_correct = False
                mismatches.append(
                    f"Row {i}, Column {header[j]}: expected '{truth_val}', got '{converted_val}'"
                )
    
    return converted_solution, is_correct, mismatches


def validate_solution(solver_solution, attrs, gridmode_solution):

    if not solver_solution:
        print("no solution found!")
        return
    converted_solution, is_correct, mismatches = convert_solver_solution_to_gridmode_format(
        solver_solution, attrs, gridmode_solution
    )
    return is_correct


In [ ]:
gridmode = pd.read_parquet("Gridmode-00000-of-00001.parquet")
mc = pd.read_parquet("mc-00000-of-00001.parquet")

skip = 0

gridmode = gridmode[skip:]

In [ ]:
def solve_test(index, verbose=False):
    ppp = PreProcess()

    provided_solution = gridmode.solution.iloc[index]
    puzzle = gridmode.puzzle.iloc[index].lower()
    attrs, clues = ppp.proccess(puzzle)


    unsolvable = 0
    for key in attrs.keys():
        if key == "pet" or key == "animals":
            unsolvable += 1
            if unsolvable == 2:
                # Check if pet and animals have overlapping values
                pet_values = set(attrs.get("pet", []))
                animals_values = set(attrs.get("animals", []))
                overlap = pet_values & animals_values  # intersection
                if overlap:
                    if verbose:
                        print(puzzle)
                        print("================================")
                    print(f"Puzzle {index} not solvable because pet and animals have overlapping values: {overlap}")
                    print(f"can not deside what value belongs to what attribute")
                    return False

    # Track constraint creation time
    constraint_start = time.time()
    constrains: list[Constraint] = constraint_factory(attrs, clues)
    constraint_time = time.time() - constraint_start

    # Track solver time
    solver_start = time.time()
    Cs = ConstraintSolver(attrs, constrains)
    solution = Cs.solve()
    solver_time = time.time() - solver_start

    if verbose:
        print(puzzle)
        print("================================")
        print("Provided solution")
        print(provided_solution)
        print("================================")
        print("Build constraints")
        for c in constrains:
            print(c.get_info())
        print("================================")
        print("Timing Information")
        print(f"Constraint creation time: {constraint_time:.4f}s")
        print(f"Solver time: {solver_time:.4f}s")
        print(f"Backtrack count: {Cs.backtrack_count}")
        print(f"Propagation calls: {Cs.propagation_calls}")
        print("================================")
        print("Our Solution")
        Cs.print_solution(solution)

    return validate_solution(solution, attrs, provided_solution)



def solve_all_tests():
    total = len(gridmode)
    passed = 0
    failed = []
    times = []
    
    start_total = time.time()
    
    for i in range(total):
        print(f"Testing {i}/{total}...", end="\r")
        test_start = time.time()
        if not solve_test(i):
            failed.append(i)
            print(f"wrong at index: {i} true index: {i+skip}")
        test_end = time.time()
        times.append(test_end - test_start)
        passed += 1
    
    total_time = time.time() - start_total
    avg_time = sum(times) / len(times) if times else 0
    
    print(f"\nResults: {passed}/{total} passed")
    if failed:
        print(f"Failed indices: {failed}")
    print(f"Total time: {total_time:.2f}s | Average time per solution: {avg_time:.4f}s")

In [ ]:
solve_test(327,verbose=True)

In [ ]:
#solve_all_tests()